In [1]:
# 導入函式庫
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense, Bidirectional
from keras.optimizers import Adam


In [2]:
# 固定亂數種子，使每次執行產生的亂數都一樣
np.random.seed(1337)


# 載入 MNIST 資料庫的訓練資料，並自動分為『訓練組』及『測試組』
(X_train, y_train), (X_test, y_test_org) = mnist.load_data()

In [3]:
# 將 training 的 input 資料轉為3維，並 normalize 把顏色控制在 0 ~ 1 之間
X_train = X_train.reshape(-1, 28, 28) / 255.      
X_test = X_test.reshape(-1, 28, 28) / 255.
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test_org, num_classes=10)

In [5]:
# 建立簡單的線性執行的模型
model = Sequential()
# 加 RNN 隱藏層(hidden layer)
model.add(Bidirectional(SimpleRNN(
    # 如果後端使用tensorflow，batch_input_shape 的 batch_size 需設為 None.
    # 否則執行 model.evaluate() 會有錯誤產生.
    batch_input_shape=(None, 28, 28), 
    units= 50,
    unroll=True,
))) 
# 加 output 層
model.add(Dense(units=10, kernel_initializer='normal', activation='softmax'))

# 編譯: 選擇損失函數、優化方法及成效衡量方式
LR = 0.001          # Learning Rate
adam = Adam(LR)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
#model.summary()        

In [6]:
# 一批訓練多少張圖片
BATCH_SIZE = 50     
BATCH_INDEX = 0     
# 訓練模型 4001 次
for step in range(1, 4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    # 逐批訓練
    loss = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX

    # 每 500 批，顯示測試的準確率
    if step % 500 == 0:
        # 模型評估
        loss, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], 
            verbose=False)
        print("test loss: {}  test accuracy: {}".format(loss,accuracy))

test loss: 0.39376118779182434  test accuracy: 0.8873999714851379
test loss: 0.25738802552223206  test accuracy: 0.9240000247955322
test loss: 0.2216954529285431  test accuracy: 0.9351999759674072
test loss: 0.20104259252548218  test accuracy: 0.9398999810218811
test loss: 0.16216619312763214  test accuracy: 0.9524000287055969
test loss: 0.16149085760116577  test accuracy: 0.954200029373169
test loss: 0.14679931104183197  test accuracy: 0.9574000239372253
test loss: 0.1598879098892212  test accuracy: 0.9527000188827515


In [8]:
# 預測(prediction)
import numpy as np
X = X_test[:,:]
predictions = np.argmax(model.predict(X), axis=1)
# get prediction result
print('actual :', y_test_org[:])
print('predict:', predictions)



actual : [7 2 1 ... 4 5 6]
predict: [7 2 1 ... 4 5 6]


In [10]:
(y_test_org==predictions).sum() / len(predictions) * 100

95.27